In [13]:
!sudo apt-get -y install tesseract
!sudo  apt-get -y install poppler-utils
!pip install pdf2image pytesseract opencv-python matplotlib


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package tesseract
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.8).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [14]:
!sudo apt-get update
!sudo apt-get install -y tesseract-ocr


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 257 kB in 1s (277 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [15]:
!sudo apt-get install -y libgl1-mesa-glx


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libdrm-amdgpu1 libdrm-common libdrm-intel1 libdrm-nouveau2 libdrm-radeon1
  libdrm2 libelf1 libgl1 libgl1-amber-dri libgl1-mesa-dri libglapi-mesa
  libglvnd0 libglx-mesa0 libglx0 libllvm15 libpciaccess0 libsensors-config
  libsensors5 libx11-xcb1 libxcb-dri2-0 libxcb-dri3-0 libxcb-glx0
  libxcb-present0 libxcb-randr0 libxcb-sync1 libxcb-xfixes0 libxfixes3
  libxshmfence1 libxxf86vm1
Suggested packages:
  pciutils lm-sensors
The following NEW packages will be installed:
  libdrm-amdgpu1 libdrm-common libdrm-intel1 libdrm-nouveau2 libdrm-radeon1
  libdrm2 libelf1 libgl1 libgl1-amber-dri libgl1-mesa-dri libgl1-mesa-glx
  libglapi-mesa libglvnd0 libglx-mesa0 libglx0 libllvm15 libpciaccess0
  libsensors-config libsensors5 libx11-xcb1 libxcb-dri2-0 libxcb-dri3-0
  libxcb-glx0 libxcb-present0 libxcb-randr0 libxcb-sync1 libxcb-xfixes0
  libxfix

In [26]:
import numpy as np
import cv2
import pytesseract
from pdf2image import convert_from_path
from PIL import Image
import re

# 📄 Chemin vers ton PDF
PDF_PATH = "ah.pdf"

# 🧾 Convertir TOUTES les pages du PDF
images = convert_from_path(PDF_PATH, dpi=300)

# 🔁 Initialiser la liste des blocs OCR
all_blocks = []

# 🧠 OCR sur chaque page
for page_num, image in enumerate(images):
    image_cv = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    custom_config = r'--oem 3 --psm 1'
    data = pytesseract.image_to_data(image_cv, config=custom_config, lang='eng+fra', output_type=pytesseract.Output.DICT)

    n_boxes = len(data['level'])
    for i in range(n_boxes):
        text = data['text'][i].strip()
        if text:
            (x, y, w, h) = (data['left'][i], data['top'][i], data['width'][i], data['height'][i])
            all_blocks.append({
                "text": text,
                "box": (x, y + page_num * 3000, w, h)  # ⚠️ Décalage Y pour garder l’ordre entre pages
            })

# 🔠 Grouper par ligne
def group_lines(blocks, y_threshold=15):
    sorted_blocks = sorted(blocks, key=lambda b: (b['box'][1], b['box'][0]))
    lines = []
    current_line = []
    last_y = None
    for blk in sorted_blocks:
        x, y, w, h = blk['box']
        if last_y is None or abs(y - last_y) <= y_threshold:
            current_line.append(blk)
        else:
            lines.append(current_line)
            current_line = [blk]
        last_y = y
    if current_line:
        lines.append(current_line)
    return lines

# 📜 Fusionner les mots par ligne
def lines_to_text(lines):
    line_texts = []
    for line in lines:
        line_sorted = sorted(line, key=lambda b: b['box'][0])
        full_line = " ".join([b['text'] for b in line_sorted])
        line_texts.append(full_line)
    return line_texts

# 📂 Regrouper par section
def group_sections(lines):
    sections = {}
    current_section = "contact"
    sections[current_section] = []

    section_keywords = {
        "profile": ["profil", "présentation", "summary"],
        "experience": ["expérience", "experience", "Expériences professionnelles"],
        "education": ["formation","Formations", "education", "diplôme"],
        "skills": ["compétence", "Compétences clés", "skills"],
        "certifications": ["certification"],
    }

    for line in lines:
        clean = line.strip().lower()

        matched = False
        for section, keywords in section_keywords.items():
            for kw in keywords:
                if re.search(rf"\b{kw}\b", clean):
                    current_section = section
                    matched = True
                    break
            if matched:
                break
        sections.setdefault(current_section, []).append(line)

    return sections

# ✅ Appliquer les étapes
lines = group_lines(all_blocks)
line_texts = lines_to_text(lines)
sections = group_sections(line_texts)

# 🖨️ Affichage final
for section, content in sections.items():
    print(f"\n==== SECTION: {section.upper()} ====")
    for line in content:
        print(line)



==== SECTION: CONTACT ====
\
Téléphone : +33 6 44 18 07 72 Achraf HADJ TAIEB
= Email : achraf.ht@gmail.com
© LinkedIn : linkedin.com/in/achrafht Directeur et Chef de projet IT

==== SECTION: EXPERIENCE ====
16 ans d’expérience
Directeur IT et Chef de projet expérimenté, certifié Agile Scrum PSPO |, avec plus de 16 ans d’expérience dans la
gestion de projets stratégiques et la conduite de transformations digitales. Reconnu pour ma rigueur, ma capacité a
piloter des équipes pluridisciplinaires et multiculturelles, j‘interviens avec efficacité sur des environnements
complexes, en garantissant un haut niveau de qualité, de performance et de conformité.
Information
Localisation : Paris - Impact Carbonne : 9.78 g COze TJM : 700€ Disponibilité : Immédiate
Compétences clés
Managériales Chefferie/Directions de projet
Leadership avéré dans la direction de projets Conduite de projets techniques complexes liés a
complexes (jusqu’a 40 pays impactés). des environnements critiques (Mainframe,
Gestio

In [21]:
import numpy as np
import re

# Étape 1 : Regrouper les blocs OCR ligne par ligne (par Y)
def group_lines(blocks, y_threshold=15):
    sorted_blocks = sorted(blocks, key=lambda b: (b['box'][1], b['box'][0]))  # tri Y, puis X
    lines = []
    current_line = []
    last_y = None

    for blk in sorted_blocks:
        x, y, w, h = blk['box']
        if last_y is None or abs(y - last_y) <= y_threshold:
            current_line.append(blk)
        else:
            lines.append(current_line)
            current_line = [blk]
        last_y = y
    if current_line:
        lines.append(current_line)
    return lines

# Étape 2 : Fusionner les mots de chaque ligne
def lines_to_text(lines):
    line_texts = []
    for line in lines:
        line_sorted = sorted(line, key=lambda b: b['box'][0])
        full_line = " ".join([b['text'] for b in line_sorted])
        line_texts.append(full_line)
    return line_texts

# Étape 3 : Regrouper les lignes par sections
def group_sections(lines):
    sections = {}
    current_section = "contact"
    sections[current_section] = []

    section_keywords = {
        "profile": ["summary", "profil"],
        "experience": ["experience", "expérience"],
        "education": ["education", "formation", "diplôme"],
        "skills": ["skills", "compétences"],
        "projects": ["projects", "projets"],
        "certifications": ["certification"]
    }

    for line in lines:
        clean = line.strip().lower()

        # détecter section
        matched = False
        for section, keywords in section_keywords.items():
            for kw in keywords:
                if re.search(rf"\b{kw}\b", clean):
                    current_section = section
                    matched = True
                    break
            if matched:
                break
        sections.setdefault(current_section, []).append(line)

    return sections


In [22]:
# Grouper les blocs OCR par ligne
lines = group_lines(blocks)

# Fusionner les mots de chaque ligne
line_texts = lines_to_text(lines)

# Regrouper les lignes par sections
sections = group_sections(line_texts)

# Afficher les sections
for section, content in sections.items():
    print(f"\n==== SECTION: {section.upper()} ====")
    for line in content:
        print(line)



==== SECTION: CONTACT ====
\
Téléphone : +33 6 44 18 07 72 Achraf HADJ TAIEB
= Email : achraf.ht@gmail.com
© LinkedIn : linkedin.com/in/achrafht Directeur et Chef de projet IT

==== SECTION: EXPERIENCE ====
16 ans d’expérience
Directeur IT et Chef de projet expérimenté, certifié Agile Scrum PSPO |, avec plus de 16 ans d’expérience dans la
Expérience dans la définition et la validation des architectures microservices.
architectures techniques (microservices, API). Maitrise des outils collaboratifs (JIRA,
Collaboration avec des parties prenantes variées Confluence) et des pratiques DevOps (CI/CD,
(techniques, métiers, clients). TDD, BDD).
Gestion des phases de tests et de recette (plan Mise en ceuvre de pratiques avancées : sécurité
de test, rapport de tests, recette utilisateur). by design, automatisation des tests, monitoring.
Familiarité avec les environnements Maintrame
et la migration d’intrastructures.
Méthodologie Secteur métier :
Meéthodes Agile (Scrum et kanban) Framework Banqu